In [0]:
# DESCRIPTION: Normalizes historical points to the modern 2024 system (25-18-15...)
# ---------------------------------------------------------

from pyspark.sql.functions import col, when, sum as _sum

# 1. Read Silver Data
df = spark.read.format("delta").load("abfss://silver@YOUR_ACCOUNT.dfs.core.windows.net/f1_results")

# 2. Apply Modern Scoring Logic
df_normalized = df.withColumn("normalized_points",
    when(col("position") == 1, 25)
    .when(col("position") == 2, 18)
    .when(col("position") == 3, 15)
    .when(col("position") == 4, 12)
    .when(col("position") == 5, 10)
    .when(col("position") == 6, 8)
    .when(col("position") == 7, 6)
    .when(col("position") == 8, 4)
    .when(col("position") == 9, 2)
    .when(col("position") == 10, 1)
    .otherwise(0)
)

# 3. Aggregate for Power BI (Drivers Page)
df_gold = df_normalized.groupBy("season", "driver_first_name", "driver_last_name", "driver_nationality") \
    .agg(_sum("normalized_points").alias("total_normalized_points")) \
    .orderBy(col("total_normalized_points").desc())

# 4. Save to Gold
df_gold.write.mode("overwrite").format("parquet").save("abfss://gold@YOUR_ACCOUNT.dfs.core.windows.net/normalized_analysis")